# Import

In [1]:
import pandas as pd
import requests
import numpy as np
import json
from bs4 import BeautifulSoup as bs
import random
import time
import json
import pickle

from os.path import join
from itertools import islice

# Scraper

Toma todos los animes a los cuales un usuario les ha dado rating, y luego encuentra sus detalles en MyAnimeList.

In [2]:
def extraer_animes(animes: set, head, parametros):
    """
    animes debe ser un set con todos los ids de los animes que se quieran extraer.

    Es recomendado guardar el output en pickles para ser compacto en el espacio.
    """
    detalles_anime = {}
    for anime in animes:
        time.sleep(3)
        detalles_anime[anime] = {}

        while True:
            try:
                response = requests.get(
                    url=f'https://api.myanimelist.net/v2/anime/{anime}',
                    headers=head,
                    params=parametros
                )

                # Manejar errores / Status codes.
                if response.status_code == 429: # Demasiadas peticiones
                    print("Demasiadas peticiones")
                    time.sleep(120)
                    continue
                elif response.status_code == 403: # prohibido.
                    print("prohibido")
                    time.sleep(120)
                    continue
                elif response.status_code == 404: # No se encontro el anime.
                    print("perdido")
                    break
                elif (response.status_code // 100) == 5: # Error en el servidor.
                    print(response.status_code)
                    print(response.json())
                    print(anime)
                    time.sleep(120)
                    continue
                elif response.status_code != 200: # Error desconocido
                    print(response.status_code)
                    print(response.json())
                    print(anime)
                    time.sleep(120)
                    continue
                detalles_anime[anime]["detalles"] = response.json()
                break
            except KeyboardInterrupt: # Usuario saltado por algun error etc.
                print("anime saltado")
                print(response.status_code)
                print(response.json())
                print(anime)
                break
            except ConnectionError:
                print("Error de internet")
                print(anime)
                time.sleep(300)
                continue
    # Estas listas idealmente deben ser guardadas en pickle para usar eficientemente el espacio.
    return detalles_anime

In [3]:
animes_unicos = set()

for i in range(1, 21):
    path = join("raw", f"listas_usuarios_{i}.pickle")
    with open(path, "rb") as archivo:
        listas = pickle.load(archivo)
    for user_id, value in listas.items():
        name = value.pop("nombre")
        tipo = value.pop("tipo")
        if tipo != "publica":
            continue
        value = {x[0] for x in value.items() if (x[1][1] == 2) and (x[1][0] > 0)}
        if len(value) == 0:
            continue
        animes_unicos = animes_unicos.union(value)
len(animes_unicos)

8978

In [4]:
url_headers = {
    "accept": "application/json",
    "X-MAL-CLIENT-ID": "dba80244ba88ffa1c4914806e9ccb3e4"
}

animedetail_params = {
    "fields": "title,alternative_titles,start_date,end_date,synopsis,mean,rank,popularity,num_list_users,num_scoring_users,media_type,status,num_episodes,start_season,studios",
}

In [9]:
cantidad = len(animes_unicos)

for i in range(7, 21):
    conj = set(islice(animes_unicos, ((i-1)*cantidad)//20, (i*cantidad)//20))
    resultado = extraer_animes(conj, url_headers, animedetail_params)

    path = join("raw", f"detalles_anime_{i}.pickle")
    with open(path, "wb") as archivo:
        pickle.dump(resultado, archivo)

In [10]:
for i in range(2, 7):
    conj = set(islice(animes_unicos, ((i-1)*cantidad)//20, (i*cantidad)//20))
    resultado = extraer_animes(conj, url_headers, animedetail_params)

    path = join("raw", f"detalles_anime_{i}.pickle")
    with open(path, "wb") as archivo:
        pickle.dump(resultado, archivo)